In [ ]:
import pickle
import sys
sys.path.insert(0, '../../../summarisation/preprocess')
import vectorizer

In [ ]:
training_data = open('preprocessed_data/beer/train.txt', 'r').readlines()

In [ ]:
training_data = [x.split() for x in training_data]

In [ ]:
from collections import defaultdict
vocab = defaultdict(int)
for line in training_data :
    for word in line :
        vocab[word] += 1

In [ ]:
filter_vocab = {k:v for k, v in vocab.items() if not any(char.isdigit() for char in k)}
sorted_vocab = sorted(filter_vocab.items(), key=lambda x:-x[1])
top10k = sorted_vocab[:10000]
vocab_10k = {k:v for k,v in top10k}

In [ ]:
word2idx = {'[0]' : 0, '<unk>' : 1, '<qqq>' : 2}
idx = len(word2idx)
for word in vocab_10k :
    word2idx[word] = idx
    idx+=1

In [ ]:
idx_train = []
for i, line in enumerate(training_data) :
    nline = []
    for word in line :
        if any(char.isdigit() for char in word) :
            nline.append(2)
        elif word in word2idx :
            nline.append(word2idx[word])
        else :
            nline.append(1)
    idx_train.append(nline)
    if i%100000 == 0:
        print(i)

In [ ]:
test_data = open('preprocessed_data/beer/test.txt', 'r').readlines()
test_data = [x.split() for x in test_data]

In [ ]:
idx_test = []
for i, line in enumerate(test_data) :
    nline = []
    for word in line :
        if any(char.isdigit() for char in word) :
            nline.append(2)
        elif word in word2idx :
            nline.append(word2idx[word])
        else :
            nline.append(1)
    idx_test.append(nline)
    if i%100000 == 0:
        print(i)

In [ ]:
idx2word = {v:k for k,v in word2idx.items()}

In [ ]:
data = {'word2idx' : word2idx, 'idx2word' : idx2word, 'training' : idx_train, 'test' : idx_test}

In [ ]:
#pickle.dump(data, open('data_beer.p', 'wb'))

In [1]:
import pickle
data = pickle.load(open('data_beer.p', 'rb'))

In [2]:
from gensim.models import Word2Vec
model = Word2Vec.load('preprocessed_data/beer/w2v_embedding')

In [3]:
import numpy as np
embeddings = np.zeros((len(data['word2idx']), 200))

In [4]:
for word in data['word2idx'] :
    if word in model.vocab :
        embeddings[data['word2idx'][word], :] = list(model[word])
        

In [5]:
for i in range(3) :
    embeddings[i, :] = np.random.randn(200)

In [6]:
data['embeddings'] = embeddings

In [7]:
pickle.dump(data, open('data_beer_embed.p', 'wb'))

In [36]:
for word in data['word2idx'] :
    if word in model.vocab :
        assert all(embeddings[data['word2idx'][word]] == list(model[word]))